# Feature Processing

En este notebook obtenemos los features de interés a partir de las matrices de mediciones de los sujetos.

### Imports

In [1]:
%pylab
%matplotlib inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function, division, unicode_literals
import glob
import os.path
import pickle
import scipy.io as sio
import scipy.signal as scs
import pandas as pd
import scipy as sp

## Definiciones globales

In [3]:
pickle_dir = 'pickles'

In [4]:
if not os.path.isdir(pickle_dir):
    os.mkdir(pickle_dir)

In [5]:
dataDir = 'dataset'
fs = 250

## Identificación de sujetos

In [6]:
SUBJECT_FILES = 'subjects_files.pkl'
subjects_files_path = os.path.join(pickle_dir, SUBJECT_FILES)

if not os.path.isfile(subjects_files_path):
    subjects = glob.glob('dataset/*.mat')
    matrix_files = sorted(subjects)  # dataset/P01.mat ...
    matrix_names = [os.path.basename(n).replace('.mat', '') for n in matrix_files]  # P01.mat ...
    
    with open(os.path.join(pickle_dir, SUBJECT_FILES), 'wb') as f:
        pickle.dump(matrix_files, f, protocol=2)
else:
    with open(os.path.join(pickle_dir, SUBJECT_FILES), 'rb') as f:
        matrix_files = pickle.load(f)  # dataset/P01.mat ...
        matrix_names = [os.path.basename(n).replace('.mat', '') for n in matrix_files]  # P01.mat ...

## Potencia en bandas

In [7]:
# Nombres de archivos

BAND_POWER_MEAN = 'band_power_mean.pkl'
BAND_POWER_STD = 'band_power_std.pkl'
BAND_POWER_NORM_MEAN = 'band_power_norm_mean.pkl'
BAND_POWER_NORM_STD = 'band_power_norm_std.pkl'
band_power_mean_file = os.path.join(pickle_dir, BAND_POWER_MEAN)
band_power_std_file = os.path.join(pickle_dir, BAND_POWER_STD)
band_power_norm_mean_file = os.path.join(pickle_dir, BAND_POWER_NORM_MEAN)
band_power_norm_std_file = os.path.join(pickle_dir, BAND_POWER_NORM_STD)

In [12]:
# Inicialización variables
band_names = ['alpha', 'beta', 'gamma', 'delta', 'theta']

band_mean = pd.DataFrame(index=matrix_names, columns=band_names)
band_norm_mean= pd.DataFrame(index=matrix_names, columns=band_names)
band_std = pd.DataFrame(index=matrix_names, columns=band_names)
band_norm_std= pd.DataFrame(index=matrix_names, columns=band_names)

In [13]:
# Calculo de información

for idx in range(len(matrix_names)):
    file = matrix_files[idx]
    print(file)
    subject_name = matrix_names[idx]
    data = sio.loadmat(file)
    data_array = data['data']

    # potencia: epoch x electrodo x freq
    freq, Pxx_trials_subjects = scs.welch(data_array[:,:,:], fs=fs)

    # potencia: epoch x freq 
    Pxx_epoch_subjects = np.mean(Pxx_trials_subjects, axis=1)

    # potencia (de banda): epoch
    Pxx_epoch_delta_subjects = np.sum(Pxx_epoch_subjects[:, freq < 4], axis=1)
    Pxx_epoch_theta_subjects = np.sum(Pxx_epoch_subjects[:, (freq >= 4)*(freq < 8)], axis=1)
    Pxx_epoch_beta_subjects = np.sum(Pxx_epoch_subjects[:, (freq >= 13)*(freq < 30)], axis=1)
    Pxx_epoch_gamma_subjects = np.sum(Pxx_epoch_subjects[:, freq >= 30], axis=1)
    Pxx_epoch_alpha_subjects = np.sum(Pxx_epoch_subjects[:, freq >= 30], axis=1)
    
    band_mean['delta'][subject_name] = Pxx_epoch_delta_subjects.mean()
    band_mean['theta'][subject_name] = Pxx_epoch_theta_subjects.mean()
    band_mean['beta'][subject_name] = Pxx_epoch_beta_subjects.mean()
    band_mean['gamma'][subject_name] = Pxx_epoch_gamma_subjects.mean()
    band_mean['alpha'][subject_name] = Pxx_epoch_alpha_subjects.mean()
    
    band_std['delta'][subject_name] = Pxx_epoch_delta_subjects.std()
    band_std['theta'][subject_name] = Pxx_epoch_theta_subjects.std()
    band_std['beta'][subject_name] = Pxx_epoch_beta_subjects.std()
    band_std['gamma'][subject_name] = Pxx_epoch_gamma_subjects.std()
    band_std['alpha'][subject_name] = Pxx_epoch_alpha_subjects.std()
    
    band_sum = np.array([
        Pxx_epoch_delta_subjects,
        Pxx_epoch_theta_subjects,
        Pxx_epoch_beta_subjects,
        Pxx_epoch_gamma_subjects,
        Pxx_epoch_alpha_subjects
    ]).sum(axis=0)
    
    band_norm_mean['delta'][subject_name] = (Pxx_epoch_delta_subjects / band_sum).mean()
    band_norm_mean['theta'][subject_name] = (Pxx_epoch_theta_subjects / band_sum).mean()
    band_norm_mean['beta'][subject_name] = (Pxx_epoch_beta_subjects / band_sum).mean()
    band_norm_mean['gamma'][subject_name] = (Pxx_epoch_gamma_subjects / band_sum).mean()
    band_norm_mean['alpha'][subject_name] = (Pxx_epoch_alpha_subjects / band_sum).mean()
    
    band_norm_std['delta'][subject_name] = (Pxx_epoch_delta_subjects / band_sum).std()
    band_norm_std['theta'][subject_name] = (Pxx_epoch_theta_subjects / band_sum).std()
    band_norm_std['beta'][subject_name] = (Pxx_epoch_beta_subjects / band_sum).std()
    band_norm_std['gamma'][subject_name] = (Pxx_epoch_gamma_subjects / band_sum).std()
    band_norm_std['alpha'][subject_name] = (Pxx_epoch_alpha_subjects / band_sum).std()

dataset/P01.mat


C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 201, using nperseg = 201
  .format(nperseg, input_length))


dataset/P02.mat
dataset/P03.mat
dataset/P04.mat
dataset/P05.mat
dataset/P06.mat
dataset/P07.mat
dataset/P08.mat
dataset/P09.mat
dataset/P10.mat
dataset/S01.mat
dataset/S02.mat
dataset/S03.mat
dataset/S04.mat
dataset/S05.mat
dataset/S06.mat
dataset/S07.mat
dataset/S08.mat
dataset/S09.mat
dataset/S10.mat


In [14]:
# Grabación
with open(band_power_mean_file, 'wb') as f:
    pickle.dump(band_mean, f, protocol=2)

with open(band_power_std_file, 'wb') as f:
    pickle.dump(band_std, f, protocol=2)

with open(band_power_norm_mean_file, 'wb') as f:
    pickle.dump(band_norm_mean, f, protocol=2)

with open(band_power_norm_std_file, 'wb') as f:
    pickle.dump(band_norm_std, f, protocol=2)

## Información intra-electrodo

In [95]:
def subject_entropy_var(subject_matrix):
    "Matriz entropía epoch x electrodo"
    H = np.zeros((subject_matrix.shape[0], subject_matrix.shape[1]))
    for i in range(subject_matrix.shape[0]):
        for j in range(subject_matrix.shape[1]):
            hist, bin_edges = np.histogram(subject_matrix[i, j, :], bins='scott')
            H [i, j] = sp.stats.entropy(hist/sum(hist))
    return H

In [93]:
INTRA_ENTROPY = 'intra_entropy.pkl'
intra_entropy_file = os.path.join(pickle_dir, INTRA_ENTROPY)

In [94]:
intra_entropy_columns = ['mean', 'std']
intra_entropy = pd.DataFrame(index=matrix_names, columns=intra_entropy_columns)

In [98]:
for idx in range(len(matrix_names)):
    file = matrix_files[idx]
    print(file)
    subject_name = matrix_names[idx]
    data = sio.loadmat(file)
    data_array = data['data']
    
    H = subject_entropy_var(data_array)  # epoch x electrodo
    H_epoch = H.mean(axis=1)  # epoch
   
    intra_entropy['mean'][subject_name] = H_epoch.mean()
    intra_entropy['std'][subject_name] = H_epoch.std()

dataset/P01.mat
dataset/P02.mat
dataset/P03.mat
dataset/P04.mat
dataset/P05.mat
dataset/P06.mat
dataset/P07.mat
dataset/P08.mat
dataset/P09.mat
dataset/P10.mat
dataset/S01.mat
dataset/S02.mat
dataset/S03.mat
dataset/S04.mat
dataset/S05.mat
dataset/S06.mat
dataset/S07.mat
dataset/S08.mat
dataset/S09.mat
dataset/S10.mat


In [99]:
with open(intra_entropy_file, 'wb') as f:
    pickle.dump(intra_entropy, f, protocol=2)

## Información inter-electrodo

Debido a el tiempo que toma calcular la información mutua entre electrodos, este feature se extrae mediante un script. El script está pensado para que ejecute en una pc remota. Como consecuencia escribe en un log su progreso.

A continuación vemos el resultado obtenido por el [script](https://github.com/m2march/ciencia_datos/blob/master/tp3/tp3_IH.py).

In [8]:
from tp3_IH import INTER_ENTROPY

inter_electrodo = pd.read_pickle(os.path.join(pickle_dir, INTER_ENTROPY))

In [9]:
inter_electrodo

,mean,std
P01,0.002989,0.001689
P02,0.003947,0.000634
P03,0.000825,0.000142
P04,0.000962,0.000267
P05,0.001089,0.000391
P06,0.000751,0.000080
P07,0.001047,0.000172
P08,0.002055,0.000370
P09,0.001761,0.000229
P10,0.001190,0.000465
